### Juan David Orjuela - Sofía Álvarez López

In [1]:
import random
import datetime
import numpy as np
import pandas as pd
import seaborn as sns
import tensorflow as tf
from keras import backend as K
import matplotlib.pyplot as plt
#from keras import regularizers
from sklearn import preprocessing 
from sklearn.pipeline import Pipeline
pd.options.mode.chained_assignment = None
from tensorflow.keras.layers import Dense 
from scipy.stats import percentileofscore
from sklearn.metrics import mean_squared_error
from tensorflow.keras.models import Sequential
from sklearn.preprocessing import StandardScaler
from keras.wrappers.scikit_learn import KerasRegressor
from sklearn.model_selection import train_test_split, GridSearchCV
from tensorflow.keras.callbacks import EarlyStopping, ReduceLROnPlateau, TensorBoard

ModuleNotFoundError: No module named 'keras'

<h1 align='center'>Laboratorio 4: Machine Learning Techniques</h1>

## Problema 2

Para los bancos es sumamente importante reconocer transacciones fraudulentas con el objetivo de que los clientes no paguen por cosas que no compraron. Con respecto a lo anterior, se recogieron datos de más de 200 mil transacciones con un porcentaje de 0.172% de transacciones fraudulentas. Dada la baja cantidad de fraudes, se le pidió a ud que creara un modelo de inteligencia artificial para poder detectar este tipo de anomalías.

Fuente de Datos: https://www.kaggle.com/mlg-ulb/creditcardfraud


### 2.1 Exploración de los datos

Kaggle nos informa que los datos corresponden a transacciones de tarjeta de crédito hechas durante dos días en Septiembre de 2013 por tarjetahabientes europeos. Es un conjunto altamente desbalanceado, con sólo el 0.172% correspondiente a transacciones fraudulentas. Por motivos de confidencialidad, los datos se han tranformado mediante un PCA (28 componentes), exceptuando "Time" y "Amount", que corresponden al tiempo en segundos entre cada transacción y la primera transacción del set de datos, y al monto de la transacción respectivamente. La variable "Class" es la variable de clasificación, y corresponde a 1 en caso de fraude o a 0 en caso contrario.

Leamos nuestros datos y miremos una muestra aleatoria de 5 transacciones:

In [ ]:

df=pd.read_csv("creditcard.csv")
df.sample(5)

Ahora, revisamos si hay algún dato nulo en nuestro conjunto de datos. Notamos que el "máximo" de nulos es 0: es decir, ¡no hay nulos en el conjuntos de datos!

In [ ]:
print('Cantidad de nulos:', df.isnull().sum().max())

Asimismo, revisamos que, en efecto, las clases que hay corresponden a 0 (transacción corriente) y 1 (fraude).

In [ ]:
print(f"Las clases presentes en el data set son: {np.unique(df['Class'])}" )

Ahora, veamos qué tan desbalanceadas están las clases. Esperamos que haya gran desbalance, pues las transacciones fraudulentas son pocas, pero pueden representan pérdidas de millones de dólares anuales.

In [ ]:
vc = df['Class'].value_counts().to_frame().reset_index()
vc['percent'] = vc["Class"].apply(lambda x : round(100*float(x) / len(df), 2))
vc = vc.rename(columns = {"index" : "Target", "Class" : "Count"})
vc

O, gráficamente, tenemos que:

In [ ]:
sns.countplot('Class', data=df, palette=['turquoise', 'lightpink'])
plt.title('Distribuciones de clase \n (0: Corriente || 1: Fraude)')
plt.show()

Son tan poquitas transacciones fraudulentas (únicamente el 0.17%) que prácticamente es despreciable en la gráfica de arriba.

Ahora, veamos una descripción estadística de las variables del PCA que se nos ha dado:

In [ ]:
df.describe()

Notamos que la media de casi todos los componentes que provienen del PCA, como esperamos por construcción, es cercana a 0, dado que todos son del orden de $10^{-15}$ y $10^{-16}$. Asimismo, podemos apreciar que la transacción promedio tarda 94.813 ms y el monto promedio por transacción es 88.3 euros. No hay mucha más información que podamos rescatar de esto, pues las 28 variables son las componentes principales del PCA realizado.  

Ahora, veamos si tenemos datos duplicados:

In [ ]:
df.duplicated().sum()

Porcentualmente, esto es:

In [ ]:
print(np.round(100*df.duplicated().sum()/df.shape[0], 2))

Y, para la clase positiva,

In [ ]:
print(np.round(100*((df["Class"]*df.duplicated()).sum())/df["Class"].sum(), 2))

Podemos ver que solo el 0.38% de los datos están duplicados, y que 3.86% de los duplicados corresponden a la clase positiva. En realidad, es un porcentaje muy bajo. Debido a que no tenemos el conjunto de datos original, a qué se refiere o cómo se tomaron los datos, es difícil saber si es un error espurio de la adquisición de los datos o si, en efecto, así fueron las transacciones. Por lo tanto, no consideramos pertinente quitar estos datos.

Ahora, veamos las distribuciones (en histogramas) de cada una de las variables que estamos estudiando. 

In [ ]:
def draw_histograms(dataframe, features, rows, cols):
    fig=plt.figure(figsize=(20,20))
    for i, feature in enumerate(features):
        ax=fig.add_subplot(rows,cols,i+1)
        dataframe[feature].hist(bins=100,ax=ax,facecolor='dodgerblue')
        ax.set_title('Distribución del feature: ' + feature)
        ax.set_yscale('log')
        ax.grid(False)
    fig.tight_layout()  
    plt.show()
draw_histograms(df,df.drop('Class', axis=1).columns,10,3)

Se graficó en escala logarítmica en y para lograr observar los valores pequeños. Para la variable del tiempo se obtiene una distribución correspondiente a una serie de tiempo de periodo de un día. Es altamente probable que los valles correspondan a las horas de la madrugada, cuando se realizan considerablemente menos transacciones. 

Como habíamos dicho previamente, note que todas las ditribuciones están centradas en 0, como es de esperarse.

In [ ]:
fig=plt.figure(figsize=(4,4))
ax=fig.add_subplot(1,1,1)
df["Amount"].hist(bins=100,ax=ax,facecolor='dodgerblue')
ax.set_yscale('log')
ax.set_xscale('log')
ax.grid(False)
fig.tight_layout() 
plt.title('Distribución log-log de la variable Amount')
plt.show()

Por el lado del monto, se observa un comportamiento cercano a una ley de potencias (se caracteriza visualmente como una recta en una gráfica log-log), similar a la ley de Zipf o correspondiente con el principio de Pareto, algo que de alguna manera se espera: la mayoría de las transacciones son de montos pequeños y hay muy pocas de montos exorbitantes. Vemos ahora la correlación entre nuestras variables:

In [ ]:
corr = df.corr()
plt.figure(figsize=(12,10))
heat = sns.heatmap(data=corr,vmin=-1.0,vmax=1.0,cmap="viridis")
plt.title('Mapa de correlación')
plt.show()

Como es de esperarse, por construcción, la correlación es cercana a cero entre los features del PCA, entonces lo realmente interesante sería observar las otras columnas. Se observa una anticorrelación interesante entre "Amount" y el V2 y un poco el V5 también. También hay una ligera anticorrelación entre "Time" y V3.

Ahora separamos nuestro conjuntos entre fraude y no-fraude y y re-escalamos usando la desviación estándar las variables que no han sido re-escaladas mediante el PCA, es decir las corresponientes a "Time" y "Amount". Luego construimos nuestro primer autoencoder. Procedemos a distribuir los datos de la clase normal entre entrenamiento y validación, y la clase de fraude entre una clase de validación y prueba. 

Usaremos los mismo Callbacks que se aplicaron en el primer problema, a saber "Early Stopping" y "Tensorboard".

### 2.2 Particiones y preprocesamiento
Lo primero que hacemos es separar el target de las features, como vemos a continuación:

In [ ]:
x = df.drop(["Class"], axis=1)
y = df["Class"].values

Ahora, separamos los datos entre normal y fraude:

In [ ]:
x_normal, x_fraude = x[y == 0], x[y == 1]

Lo siguiente que hacemos es partir el conjunto de datos entre entrenamiento y validación. Como vamos a construir un autoencoder para la reconstrucción de anomalías, lo que debemos hacer es entrenar únicamente con los datos normales. Note que también tomamos un conjunto para realizar la validación de nuestro modelo (i.e. de la reconstrucción de transacciones normales).

No obstante, también partimos el conjunto de fraudes en dos. <code>X_fraude_train</code> lo utilizamos para ver (aún en entrenamiento) si efectivamente se detectan como anomalías las transacciones fraudulentas.

In [ ]:
X_train, X_val, y_train, y_val = train_test_split(x_normal, np.ones(len(x_normal)), test_size=0.2, random_state=28)
X_fraude_train, X_fraude_test, y_fraude_train, y_fraude_test = train_test_split(x_fraude, np.ones(len(x_fraude)), test_size=0.2, random_state=28)

Ahora, seguimos con el preprocesamiento. Lo primero que hacemos es estandarizar (usando un StandardScaler()) las dos variables que no fueron escaladas durante el PCA: Amount y Time.

In [ ]:
# Para x_train
X_train['Time'] = StandardScaler().fit_transform(X_train['Time'].values.reshape(-1, 1))
X_train['Amount'] = StandardScaler().fit_transform(X_train['Amount'].values.reshape(-1, 1))
# Para X_fraude_train
X_fraude_train['Time'] = StandardScaler().fit_transform(X_fraude_train['Time'].values.reshape(-1, 1))
X_fraude_train['Time'] = StandardScaler().fit_transform(X_fraude_train['Time'].values.reshape(-1, 1))

# Para x_val
X_val['Time'] = StandardScaler().fit_transform(X_val['Time'].values.reshape(-1, 1))
X_val['Amount'] = StandardScaler().fit_transform(X_val['Amount'].values.reshape(-1, 1))

# Los de test los haremos antes de despliegue.

### 2.3 Modelamiento
#### 2.3.1 Primer modelo baseline

Construiremos nuestros autoencoders usando una arquitectura que va reduciendo paulatinamente el número de neuronas, pasando de 30 a 16 y luego a 8 (capa latente), para luego aumentar de nuevo a 16 y 30, cuando reconstruye la entrada. Primeramente exploraremos activación de tangente hiperbólica para las capas ocultas intermedias con activación ReLu para la capa latente y la capa de salida. También probaremos exclusivamente activación ReLu.

In [ ]:
input_size = len(X_train.columns)
#Inicializamos el modelo
autoencoder = Sequential(name="Autoencoder_1")

#Encoder
autoencoder.add(Dense(30, activation='tanh', input_dim = input_size)) 
autoencoder.add(Dense(16, activation='tanh'))
autoencoder.add(Dense(8, activation='relu'))

#Decoder
autoencoder.add(Dense(16, activation='tanh'))
autoencoder.add(Dense(30, activation='tanh'))
autoencoder.add(Dense(input_size, activation='relu'))

Ahora compilamos el modelo. Debido a que tenemos un autoencoder (i.e. una tarea de regresión), usamos como función de pérdida el error cuadrático medio. Asimismo, por las mismas razones que justificamos en el problema 1, utilizamos el optimizador Adam. Finalmente, como métrica utilizamos el mismo error cuadrático medio (ya que este usamos como función de pérdida y es una de las mejores métricas en problemas de regresión).

In [ ]:
autoencoder.compile(loss='mean_squared_error', optimizer='adam', metrics=['mean_squared_error'])

Veamos cómo va nuestro modelo:

In [ ]:
autoencoder.summary()

De manera similar al problema 1, definimos y utilizamos los mismos callbacks:

In [ ]:
early_stopping = EarlyStopping(monitor='loss', patience=3, verbose=1, mode='auto', baseline=None, restore_best_weights=False)
tensorboard_callback = TensorBoard(log_dir="logs/fit/" + datetime.datetime.now().strftime("%Y%m%d-%H%M%S"), histogram_freq=0, write_graph=True, write_images=False, update_freq='epoch', profile_batch=2, embeddings_freq=0, embeddings_metadata=None,)
callbacks = [early_stopping,tensorboard_callback]

Ahora sí, hacemos fit a nuestro autoencoder. Tomamos un batch de 1000 (tan grande como sea posible).

In [ ]:
history_1 = autoencoder.fit(X_train, X_train, batch_size = 1000, epochs = 100, shuffle = True, callbacks=callbacks)    

Ya que entrenamos el modelo, podemos ver el error cuadrático medio para cada característica. Para poder visualizar esto, utilizamos el método <code>predict</code> sobre los datos de entrenamiento y validación. Para los de entrenamiento, lo que queremos, en realidad, es la salida de la última capa de la red que acabamos de construir. Los de validación no los ha visto el modelo y nos permitirá ver si se está haciendo overfitting sobre los datos de entrenamiento. 

Asimismo, queremos ver el error cuadrático medio sobre las instancias que queremos clasificar como anómalas (i.e. los fraudes). Estas también las predecimos y les calculamos el error cuadrático medio.

In [ ]:
norm_train_recons=autoencoder.predict(X_train)
norm_val_recons=autoencoder.predict(X_val)
fraud_recons=autoencoder.predict(X_fraude_train)

Calculamos el MSE. Esto nos da un dataframe con el MSE para cada feature, para train, validación y los fraudes de entrenamiento.

In [ ]:
mse_train = (np.square(norm_train_recons - X_train))
mse_val = (np.square(norm_val_recons - X_val))
mse_fraud = (np.square(fraud_recons - X_fraude_train))

Para observar la capacidad del autoencoder de reconstruir los features, obervamos la distribución del error cuadrático para cada componente. En las gráficas a continuación, los histogramas rosados corresponden a validación, los turquesa a entrenamiento y los azules a fraude.

In [ ]:
def draw_histograms(dataframe1, dataframe2, dataframe3, features, rows, cols):
    fig=plt.figure(figsize=(20,120))
    for i, feature in enumerate(features):
        ax=fig.add_subplot(rows,cols,i+1)
        dataframe3[feature].hist(bins=100,ax=ax,facecolor='dodgerblue',alpha=0.6,density=True)
        dataframe1[feature].hist(bins=100,ax=ax,facecolor='lightpink',alpha=0.6,density=True)
        dataframe2[feature].hist(bins=100,ax=ax,facecolor='turquoise',alpha=0.6,density=True)
        ax.set_title("MSE para el feature " + feature)
        ax.set_yscale('log')
        plt.ylabel('Frecuencia')
        plt.xlabel('MSE')
        ax.grid(False)
    fig.tight_layout()  
    plt.show()

draw_histograms(mse_train, mse_val, mse_fraud, mse_train.columns,30,1)

Vemos que en general pareciera que el autoencoder reconstruye mejor los componentes para la clase de transacciones normales: para la mayoría de features, tanto en entrenamiento como en validación, no sólo los errores obtenidos son similares (como esperamos de una situación de no overfitting) sino que, además, son bajos con respecto a los de transacciones fraudulentas.

Desde aquí podemos ver que, a priori, hay algunas features que ayudan a detectar mejor las transacciones fraudulentas que otras. Esto pues una mayor cantidad de transacciones fraudulentas terminó con errores cuadráticos medios bastante mayores. Estas son: Time, V9, V10, V11, V12, V14, V16, V17, V18 y Amount.

Veamos entonces la distribución del error cuadrático total sobre el conjunto de entrenamiento y validación de la clase mayoritaria y el conjunto de validación de la clase de fraude. El código de color es el mismo de arriba. Rosado para validación (i.e. transacciones corrientes con las que el modelo no fue entrenado), turquesa para entrenamiento y azul para datos fraudulentos.

In [ ]:
fig=plt.figure(figsize=(20,4))
ax=fig.add_subplot(1,1,1)
mse_fraud.sum(axis=1).hist(bins=1000,ax=ax,facecolor='dodgerblue',alpha=0.6,density=True,label='Fraude')
mse_val.sum(axis=1).hist(bins=1000,ax=ax,facecolor='lightpink',alpha=0.6,density=True,label='Validación')
mse_train.sum(axis=1).hist(bins=1000,ax=ax,facecolor='turquoise',alpha=0.6,density=True,label='Entrenamiento')
plt.ylabel('Frecuencia')
plt.xlabel('MSE')
plt.yscale('log')
plt.xscale('log')
ax.grid(False)
plt.legend()
plt.show()

Para poder visualizar bien, hacemos una gráfica log-log. Note que, en promedio, los errores asociados a la clase mayoritaria (tanto en entrenamiento como en validación) son bastante pequeños (y siguen la misma tendencia - no estamos haciendo overfitting -) comparados con los de las transacciones fraudulentas. Más aún note que, a partir de cierto valor (aproximadamente $10^2 = 100$), la cantidad de transacciones fraudulentas es mayor que las no fraudulentas. Si determináramos este como nuestro threshold, entonces definitivamente tenemos un gran rango hacia arriba de MSE que nos permite catalogar una transacción como fraudulenta.  

Ahora, podemos ver que el porcentaje de transacciones no fraudulentas cuyo error cuadrático medio en entrenamiento es mayor a 100 es:

In [ ]:
print('En entrenamiento: ','{} %'.format(np.round(100*len(mse_train[mse_train.sum(axis=1) >= 100])/len(mse_train),2)))

print('En validación: ','{} %'.format(np.round(100*len(mse_val[mse_val.sum(axis=1) >= 100])/len(mse_val),2)))

Apenas un 0.06% de diferencia: otro argumento para decir que no está haciendo overfitting.  En cambio, el porcentaje de transacciones fraudulentas con MSE mayor a 100 es:

In [ ]:
print('Fraudulentas: ','{} %'.format(np.round(100*len(mse_fraud[mse_fraud.sum(axis=1) >= 100])/len(mse_fraud),2)))

Vemos que, si definimos el corte en MSE$=100$, el 90.84% de las transacciones fraudulentas se catalogan como tal. Este es un porcentaje bastante bueno, considerando que, además, menos del 2% de las transacciones no fraudulentas se están clasificando como fraudulentas (falsos positivos). 

#### 2.3.2 Segundo modelo baseline
Nuestro segundo modelo baseline considera el uso de funciones de activación ReLu en todas sus capas.

In [ ]:
input_size = len(X_train.columns)
#Inicializamos el modelo
autoencoder_2 = Sequential(name="Autoencoder_2")

#Encoder
autoencoder_2.add(Dense(30, activation='relu', input_dim = input_size)) 
autoencoder_2.add(Dense(16, activation='relu'))
autoencoder_2.add(Dense(8, activation='relu'))

#Decoder
autoencoder_2.add(Dense(16, activation='relu'))
autoencoder_2.add(Dense(30, activation='relu'))
autoencoder_2.add(Dense(input_size, activation='relu'))

Escogemos la misma métrica, optimizador y pérdida:

In [ ]:
autoencoder_2.compile(loss='mean_squared_error', optimizer='adam', metrics=['mean_squared_error'])

In [ ]:
autoencoder_2.summary()

Ahora sí, hacemos fit a nuestro autoencoder. Tomamos un batch de 500 (tan grande como sea posible).

In [ ]:
history_2 = autoencoder_2.fit(X_train, X_train, batch_size = 500, epochs = 100, shuffle = True, callbacks=callbacks)    

Veamos la distribución del error cuadrático medio para este modelo:

In [ ]:
norm_train_recons_2 = autoencoder_2.predict(X_train)
norm_val_recons_2 = autoencoder_2.predict(X_val)
fraud_recons_2 = autoencoder_2.predict(X_fraude_train)

mse_train_2 = (np.square(norm_train_recons_2 - X_train))
mse_val_2 = (np.square(norm_val_recons_2 - X_val))
mse_fraud_2 = (np.square(fraud_recons_2 - X_fraude_train))

fig=plt.figure(figsize=(51,10))
ax=fig.add_subplot(1,1,1)
mse_val_2.sum(axis=1).hist(bins=1000,ax=ax,facecolor='lightpink',alpha=0.6,density=True,label='Validación')
mse_fraud_2.sum(axis=1).hist(bins=1000,ax=ax,facecolor='dodgerblue',alpha=0.6,density=True,label='Fraude')
mse_train_2.sum(axis=1).hist(bins=1000,ax=ax,facecolor='turquoise',alpha=0.6,density=True,label='Entrenamiento')
ax.grid(False)
plt.ylabel('Frecuencia')
plt.xlabel('MSE')
plt.yscale('log')
plt.xscale('log')
plt.legend()
plt.show()

Podemos ver que, similar al caso anterior, el comportamiento de los datos no fraudulentos en validación (rosado) y entrenamiento (turquesa) es bastante parecido. Asimismo, alrededor de MSE $\approx 100$ podemos ver que la cantidad de transacciones fraudulentas supera las no fraudulentas, similar al primer modelo.

Ahora, podemos ver que el porcentaje de transacciones no fraudulentas cuyo error cuadrático medio en entrenamiento es mayor a 100 es:

In [ ]:
print('En entrenamiento: ','{} %'.format(np.round(100*len(mse_train_2[mse_train_2.sum(axis=1) >= 100])/len(mse_train_2),2)))

print('En validación: ','{} %'.format(np.round(100*len(mse_val_2[mse_val_2.sum(axis=1) >= 100])/len(mse_val_2),2)))

Apenas un 0.07% de diferencia. Confirmamos que no hay overfitting. En cambio, el porcentaje de transacciones fraudulentas con MSE mayor a 100 es:

In [ ]:
print('Fraudulentas: ','{} %'.format(np.round(100*len(mse_fraud_2[mse_fraud_2.sum(axis=1) >= 100])/len(mse_fraud_2),2)))

Vemos que, si definimos MSE$=100$, el 91.1% de las transacciones fraudulentas se catalogan como tal. Este es un porcentaje bastante bueno.

#### 2.3.3 Modelo con ajuste de hiperparámetros
Cuando estábamos creando estos modelos, pensamos mucho en cómo hacer el ajuste de hiperparámetros; pues, en realidad, no hay muchos y los que ajustábamos antes (como cantidad de capas y número de neuronas) son más difíciles de ajustar para un autoencoder. Por lo tanto, decidimos lo siguiente: consideraremos 30 neuronas por capa en la entrada, una segunda capa de 16 y (en algunos casos) una tercera capa de 8 neuronas para el encoder. Quisimos quitar esta última capa en algunos modelos porque quizá estamos reduciendo mucho la dimensionalidad del data set. No quisimos cambiar el número de neuronas por capa por la dimensión del espacio de entrada.

Los otros hiperparámetros que decidimos ajustar fueron las funciones de activación: relu, tanh y sigmoid para las capas interiores de la red, exclusivamente ReLu para la de salida, dado el rango de valores que tiene). Note que la función de activación de las capas del centro siempre es relu. De esta forma:

In [ ]:
def entrenar_red(small_layer=True, activation_inside='relu', activation_small='relu',activation_out='relu'):
    input_size = len(X_train.columns) 
    clf = Sequential(name='Autoencoder_CV')
    clf.add(Dense(30, activation=activation_inside, input_dim = input_size))
    if small_layer:
        clf.add(Dense(16, activation=activation_inside))
        clf.add(Dense(8, activation=activation_small))
    else:
        clf.add(Dense(16, activation=activation_small))
        
    clf.add(Dense(16, activation=activation_inside))
    clf.add(Dense(30, activation=activation_inside))
    clf.add(Dense(30, activation=activation_out))
    
    clf.compile(loss='mean_squared_error', optimizer='adam', metrics=['mean_squared_error'])
    
    return clf

# Modelo que utiliza el GridSearch --> Usamos Keras Regressor y no Classifier por ser autoencoder.
modelCV_autoencoder = KerasRegressor(build_fn=entrenar_red, epochs=100, batch_size=500,verbose=1) # Modelo esqueleto

In [ ]:
pipe = Pipeline([('autoenconder_CV', modelCV_autoencoder)]) # Creamos la pipeline

# Tomamos la grilla de hiperparametros 
param_grid = dict(
                  autoenconder_CV__small_layer = [True, False],
                  autoenconder_CV__activation_inside = ['relu', 'tanh','sigmoid'],
                  autoenconder_CV__activation_out = ['relu'],
                  )


# Creamos la grilla
# Podriamos usar mas iteraciones en el CV, pero lo intentamos correr y se murio el kernel.
# Por eso lo redujimos
# grid_3 = RandomizedSearchCV(pipe_3, param_grid, verbose=3, cv=3, n_iter=20, random_state=28)
grid = GridSearchCV(pipe, param_grid=param_grid,scoring='neg_mean_squared_error',cv=3,verbose=3)

Para la CV, unimos los conjuntos de train y validación:

In [ ]:
X_joined = pd.concat([X_train, X_val])

In [ ]:
grid.fit(X_joined, X_joined, autoenconder_CV__callbacks=callbacks)

Revisamos cuál es el mejor modelo:

In [ ]:
best_model_autoencoder = grid.best_estimator_
best_model_autoencoder['autoenconder_CV'].model.summary()

# TO DO: ¿Cuáles funciones de activación?

¡Podemos ver que se eligió el modelo sin la capa de 8 neuronas! Veamos el resultado del MSE

In [ ]:
best_score_autoencoder = np.abs(grid.best_score_)
print('El MSE del mejor modelo para el autoencoder es: ', best_score_autoencoder)

De igual forma que para los otros modelos, graficamos el MSE para los datos de entrenamiento y comparamos con los de validación, con el fin de revisar que no haya overfitting (i.e. obteniendo distribuciones similares). Asimismo, revisamos con las transacciones fraudulentas a ver si no se están reconstruyendo bien (i.e. son anómalas).

In [ ]:
norm_train_recons_3 = best_model_autoencoder.predict(X_train)
norm_val_recons_3 = best_model_autoencoder.predict(X_val)
fraud_recons_3 = best_model_autoencoder.predict(X_fraude_train)

mse_train_3 = (np.square(norm_train_recons_3 - X_train))
mse_val_3 = (np.square(norm_val_recons_3 - X_val))
mse_fraud_3 = (np.square(fraud_recons_3 - X_fraude_train))

fig=plt.figure(figsize=(51,10))
ax=fig.add_subplot(1,1,1)
mse_val_3.sum(axis=1).hist(bins=1000,ax=ax,facecolor='lightpink',alpha=0.6,density=True,label='Validación')
mse_fraud_3.sum(axis=1).hist(bins=1000,ax=ax,facecolor='dodgerblue',alpha=0.6,density=True,label='Fraude')
mse_train_3.sum(axis=1).hist(bins=1000,ax=ax,facecolor='turquoise',alpha=0.6,density=True,label='Entrenamiento')
ax.grid(False)
plt.ylabel('Frecuencia')
plt.xlabel('MSE')
plt.yscale('log')
plt.xscale('log')
plt.legend()
plt.show()

Podemos ver una tendencia similar a las dos gráficas estudiadas previamente: tanto entrenamiento como validación siguen una tendencia similar en su MSE y la cantidad de transacciones fraudulentas supera a las no fraudulentas alrededor de MSE $\approx100$.

Asimismo, podemos ver que el porcentaje de transacciones no fraudulentas cuyo error cuadrático medio en entrenamiento es mayor a 100 es:

In [ ]:
print('En entrenamiento: ','{} %'.format(np.round(100*len(mse_train_3[mse_train_3.sum(axis=1) >= 100])/len(mse_train_3),2)))

In [ ]:
print('En validación: ','{} %'.format(np.round(100*len(mse_val_3[mse_val_3.sum(axis=1) >= 100])/len(mse_val_3),2)))

¡Apenas un 0.05% de diferencia! En cambio, el porcentaje de transacciones fraudulentas con MSE mayor a 100 es:

In [ ]:
print('Fraudulentas: ','{} %'.format(np.round(100*len(mse_fraud_3[mse_fraud_3.sum(axis=1) >= 100])/len(mse_fraud_3),2)))

El 87.02% de las transacciones fraudulentas se cataloga como tal si definimos el threshold en MSE$ = 100$. Esto es un número bastante bueno. Podemos incluso aumentarlo si disminuimos un poco el MSE, considerando que apenas $\approx 1.6$% de las transacciones no fraudulentas se catalogan como fraudulentas.

#### 2.3.4 Definición de los thresholds y escogencia del modelo

Podemos redefinir el threshold de MSE=$100$, a aproximadamente el 2% de las transacciones no fraudulentas catalogadas como fraudulentas; pues, como vimos previamente, son _casi_ equivalentes. De esta forma, tenemos que los porcentajes de transacciones fraudulentas catalogadas adecuadamente son:

In [ ]:
print('Modelo baseline 1: ', '{} %'.format(np.round(100 - percentileofscore(mse_fraud.sum(axis=1), np.percentile(mse_train.sum(axis=1),98, interpolation="linear"), kind="mean"),2))) 

In [ ]:
print('Modelo baseline 2: ', '{} %'.format(np.round(100 - percentileofscore(mse_fraud_2.sum(axis=1), np.percentile(mse_train_2.sum(axis=1),98, interpolation="linear"), kind="mean"),2))) 

In [ ]:
print('Modelo con CV: ', '{} %'.format(np.round(100 - percentileofscore(mse_fraud_3.sum(axis=1), np.percentile(mse_train_3.sum(axis=1),98, interpolation="linear"), kind="mean"),2))) 

En general, podemos ver que los tres porcentajes obtenidos son bastante similares: apenas difieren en máximo 1%. No obstante, el modelo que parece dar mejores resultados es el baseline 1, seguido del que tuneamos con cross validation. Por lo tanto, en un escenario en que tuviéramos que elegir uno sólo, escogeríamos el baseline 1. No obstante, también nos gustaría ver qué pasa con el de CV (pues se eligió uno con una capa menos que el baseline, con una tasa de detección de transacciones fraudulentas bastante similar.

Lo que queremos ahora es maximizar la cantidad de transacciones fraudulentas, así se nos cataloguen unas pocas (2%) transacciones normales como fraudulentas (i.e. falsos positivos). Seguramente, si aumentamos la tolerancia del 2% a, por ejemplo, el 5%, incrementaremos la cantidad de transacciones fraudulentas correctamente clasificadas (pero estaremos sacrificando las no fraudulentas). No obstante, el 5% puede ser un valor adecuado de threshold. Veamos qué pasa si tomamos este threshold para el modelo baseline 1 y el que tuneamos con CV:

In [ ]:
print('Modelo baseline 1: ', '{} %'.format(np.round(100 - percentileofscore(mse_fraud.sum(axis=1), np.percentile(mse_train.sum(axis=1),95, interpolation="linear"), kind="mean"),2))) 

In [ ]:
print('Modelo con CV: ', '{} %'.format(np.round(100 - percentileofscore(mse_fraud_3.sum(axis=1), np.percentile(mse_train_3.sum(axis=1),95, interpolation="linear"), kind="mean"),2))) 

Si aumentamos la tolerancia, parece que el modelo que tuneamos con cross validation tiene mejor reconocimiento de transacciones fraudulentas. Esto sucede cuando apenas el 5% (i.e. 11373 transacciones normales) se catalogan como fraudulentas, y se nos escapan apenas 15 transacciones fraudulentas (3.82%). ¡Es un buen indicio! Probemos ahora el despliegue para ver qué tan bien está nuestro modelo.

### 2.4 Despliegue del modelo

Recordemos que el test es únicamente de transacciones fraudulentas. ¡Veamos qué tan bien quedaron clasificadas! Antes de eso, debemos escalar las variables de Amount y Time con el <code>StandardScaler()</code>, como hicimos en entrenamiento y validación.

Note que aquí no tenemos transacciones normales, pues realmente ya vimos que todos los modelos se comportan bien con ellas, como vimos previamente en los conjuntos de validación que destinamos. Pensamos que en despliegue es únicamente relevante revisar transacciones fraudulentas que el modelo no haya visto aún. 